In [1]:
import os
os.environ['autocnet_config'] = '/home/jlaura/autocnet_projects/demo.yml'
%pylab inline
from autocnet_server.graph.graph import NetworkCandidateGraph

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Create the candidate graph
ncg = NetworkCandidateGraph.from_database()

### Ring Matching
Here we apply the ring matcher to all 72 edges in the 13 image demo graph

In [4]:
ncg.apply('ring_match', on='edge', ring_radius=30, max_radius=300, target_points=25, tolerance_val=0.1)

72

#### Matches DataFrame
The matches data frame contains all of the information about matches between two images.  In this example, we are only printing the first 5 matches to the notebook.  Note how many of the columns are None. These columns can be populated by using a number of helper functions based on what information we might require.

In [25]:
ncg.edges[(1,6)]['data'].matches.head(5)

,source,source_idx,destination,destination_idx,lat,lon,geom,source_x,source_y,destination_x,destination_y,shift_x,shift_y,original_destination_x,original_destination_y
id,,,,,,,,,,,,,,,
858,1,155073,6,184233,None,None,None,None,None,None,None,None,None,None,None
859,1,107482,6,102293,None,None,None,None,None,None,None,None,None,None,None
860,1,144010,6,128210,None,None,None,None,None,None,None,None,None,None,None
861,1,16473,6,41763,None,None,None,None,None,None,None,None,None,None,None
862,1,2333,6,11608,None,None,None,None,None,None,None,None,None,None,None


#### Adding image x/y coordinates
Below, the image x/y coordinates are added to the matches dataframe. This is a nice to have, so that we do not have to do database style JOINS to get the pixel coordinates.

Since autocnet server is asynchronously submitting jobs, the cell below will initially show that the source_x, source_y, destination_x, and destination_y are None. Those value are in fact None while the jobs are asynchronously executed. After the jobs finish, comment out the apply line and rerun the print line.

In [3]:
#ncg.apply('add_coordinates_to_matches', on='edge')
ncg.edges[(1,6)]['data'].matches.head(5)

,source,source_idx,destination,destination_idx,lat,lon,geom,source_x,source_y,destination_x,destination_y,shift_x,shift_y,original_destination_x,original_destination_y
id,,,,,,,,,,,,,,,
865,1,132775,6,102454,8.60924,156.448,01010000a08c7e0e003d1657005a8e6340613e7094ee37...,138.501,6922.31,4077.16,8958.32,None,None,None,None
864,1,134144,6,103709,8.17082,156.496,01010000a08c7e0e00fefcf485e08f63403d4554147657...,92.4127,7167.61,4031.85,9204.7,None,None,None,None
877,1,189176,6,172045,8.82584,156.479,01010000a08c7e0e00464f0c42538f6340d6b5b725d5a6...,737.254,6906.5,4949.61,14490.3,None,None,None,None
885,1,162178,6,207278,8.67257,156.496,01010000a08c7e0e00fabd6c47dc8f634021a212285b58...,721.096,10833.1,4228.49,11812,None,None,None,None
890,1,225363,6,210801,8.64935,156.479,01010000a08c7e0e000ea8d6b7538f6340388e9303774c...,514.123,9774.68,4060.72,12731.2,None,None,None,None


#### Projecting the Matches to lat/lon space
Frequently, it is nice to be able to visualize the matches (for example in QGIS) for that each match needs to have an associated geometry. Below, we project the matches from pixel space into lat/lon space. This populates the lat, lon, and geom columns. A GIS uses the geom column for visualization.

In [6]:
ncg.apply('project_matches', on='edge', args=(3396190, 3376200,), srid=949900)
ncg.edges[(1,6)]['data'].matches.head(5)

,source,source_idx,destination,destination_idx,lat,lon,geom,source_x,source_y,destination_x,destination_y,shift_x,shift_y,original_destination_x,original_destination_y
id,,,,,,,,,,,,,,,
865,1,132775,6,102454,8.98404,156.477,01010000a08c7e0e00a0f83cc3468f6340cafd9593d4f7...,138.501,4830.19,4077.16,8958.32,None,None,None,None
864,1,134144,6,103709,8.60924,156.448,01010000a08c7e0e003d1657005a8e6340613e7094ee37...,92.4127,499.154,4070.11,4658.26,None,None,None,None
866,1,125904,6,129096,8.17082,156.496,01010000a08c7e0e00fefcf485e08f63403d4554147657...,931.257,6922.31,4844.24,10729.9,None,None,None,None
887,1,195736,6,234310,8.82272,156.468,01010000a08c7e0e00c4b82be2f78e63407b9275a63ba5...,316.191,7305.34,4949.61,3785.99,None,None,None,None
882,1,161293,6,146127,8.82957,156.497,01010000a08c7e0e00d4281b4ce58f63402cbde8c0bda8...,1022.26,6551.36,4286.91,7459.05,None,None,None,None


#### Data Backend
The data from an AutoCNet server project is stored in a PostGreSQL database. When matches are visualized or an edge attribute accessed, the code is hitting the database and returning either some small piece of information (a scalar or two as is the case for the ring size or a larger chunk of information such as is the case for the matches dataframe).

The PGAdmin4 tool is available to look at the data inside the database at autocnet.wr.usgs.gov (internal only).